# Google Drive ZIP to GitHub Repository Exporter

This Colab notebook provides a streamlined workflow to take a ZIP file from your Google Drive and push its contents into a new or existing GitHub repository.

### **What This Tool Does**
It automates the entire process of getting source code or other project files from a Drive link into a version-controlled repository on GitHub.

### **Required Preparation**
Before you begin, you will need three things:
1.  **A Google Drive Link:** A sharable link to the `.zip` file you want to upload. This can be a standard Drive link or a link from an application like AI Studio.
2.  **A GitHub Repository:** A repository (either new or existing, public or private) on GitHub to push the files to.
3.  **A GitHub Personal Access Token (PAT):** A PAT with `repo` scope is required for the notebook to authenticate with GitHub.

### **The Three-Step Workflow**
1.  **Step 1: Get File from Google Drive**
    *   You will provide the link to your file.
    *   The notebook authenticates with your Google account and downloads the file's content into the Colab environment.

2.  **Step 2: Unpack the ZIP Archive**
    *   This step verifies the downloaded file is a valid ZIP archive.
    *   It then unpacks the contents into a clean folder, ready to be committed to Git.

3.  **Step 3: Push Contents to GitHub Repository**
    *   You will provide your GitHub details and the name of a Colab Secret (🔑) where your PAT is stored.
    *   The notebook clones your repository, copies the project files into it, and pushes the new commit back to GitHub.

In [ ]:
# @title Step 1: Get File from Google Drive
# @markdown Paste the sharable link for your file from Google Drive.
# @markdown <br>This can be a standard ZIP file link or an AI Studio link.

# --- Configuration ---
gdrive_url = "" #@param {type:"string"}

# --- Library Imports ---
import re
import os
import shutil
import urllib.parse
import json
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io

# --- Main Logic ---
try:
    # 1. Authenticate the user.
    print("Authenticating with Google...")
    auth.authenticate_user()
    print("✅ Authentication successful.")

    # 2. Build the Google Drive service object.
    drive_service = build('drive', 'v3')

    # 3. Intelligent URL Parsing
    if not gdrive_url:
        raise ValueError("The 'gdrive_url' is empty. Please paste your URL in the field above.")

    file_id = None

    # First, try to parse it as an AI Studio URL
    if "aistudio.google.com" in gdrive_url:
        try:
            parsed_url = urllib.parse.urlparse(gdrive_url)
            query_params = urllib.parse.parse_qs(parsed_url.query)
            state_json = query_params['state'][0]
            state_data = json.loads(state_json)
            if state_data.get('ids'):
                file_id = state_data['ids'][0]
                print("Detected AI Studio URL format.")
        except (KeyError, IndexError, json.JSONDecodeError):
            # It looked like an AI Studio URL but failed to parse, pass to the next method
            pass

    # If it's not a valid AI Studio URL, try the standard Google Drive format
    if not file_id:
        match = re.search(r'/d/([a-zA-Z0-9_-]+)', gdrive_url)
        if match:
            file_id = match.group(1)
            print("Detected standard Google Drive URL format.")

    # If neither method worked, raise an error
    if not file_id:
        raise ValueError("Could not find a valid File ID. Please provide a standard Google Drive or AI Studio sharable link.")

    print(f"🔎 Parsed File ID: {file_id}")

    # 4. Get file metadata (like the title)
    print(f"⬇️  Fetching file metadata from Google Drive...")
    file_metadata = drive_service.files().get(fileId=file_id, fields='name, id').execute()
    file_name = file_metadata.get('name')
    print(f"📄 File Name: '{file_name}'")

    # 5. Download the file content
    request = drive_service.files().get_media(fileId=file_id)
    file_content_stream = io.BytesIO()
    downloader = MediaIoBaseDownload(file_content_stream, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f"   Downloading... {int(status.progress() * 100)}%")

    # Store the downloaded content for the next cell
    gdrive_file_bytes = file_content_stream.getvalue()
    print("\n✅ File content downloaded successfully.")

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

In [ ]:
# @title Step 2: Unpack the ZIP Archive

import zipfile

print("--- Unpacking Archive ---")

try:
    if 'gdrive_file_bytes' not in locals() or not gdrive_file_bytes:
        raise ValueError("File content not found. Please run Step 1 successfully first.")

    # Confirm it's a zip file before proceeding
    if not gdrive_file_bytes.startswith(b'PK\x03\x04'):
        raise TypeError("The downloaded file is not a ZIP archive. Please provide a link to a valid .zip file.")

    # Define paths for our operations
    temp_zip_path = "original_archive.zip"
    extraction_path = "project_content" # This is the generic folder for the repo contents

    # Clean up from any previous runs
    if os.path.exists(extraction_path):
        shutil.rmtree(extraction_path)
    if os.path.exists(temp_zip_path):
        os.remove(temp_zip_path)

    # 1. Save the downloaded bytes to a temporary zip file
    with open(temp_zip_path, 'wb') as f:
        f.write(gdrive_file_bytes)
    print(f"📦 Saved downloaded data to '{temp_zip_path}'")

    # 2. Unzip the archive to see its contents
    os.makedirs(extraction_path, exist_ok=True)
    with zipfile.ZipFile(temp_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)

    extracted_files = os.listdir(extraction_path)
    print(f"✅ Successfully extracted {len(extracted_files)} file(s) into the '{extraction_path}' folder.")

    # 3. Clean up the temporary zip file
    os.remove(temp_zip_path)

    print(f"\n✅ Success! The project contents are ready in the '{extraction_path}' directory.")
    print("   You can now proceed to the next cell to push the contents to GitHub.")

except Exception as e:
    print(f"\n❌ An error occurred during the unpacking process: {e}")

In [ ]:
# @title Step 3: Push Contents to GitHub Repository

#@markdown ### ⬇️ Fill in your GitHub details below
#@markdown ---
#@markdown **Instructions:**
#@markdown 1. Go to GitHub and create a new repository (Public or Private). You can initialize it with a README.
#@markdown 2. Create a Personal Access Token with `repo` scope.
#@markdown 3. **Store your PAT as a Colab Secret (🔑).**
#@markdown 4. Fill in the repository and secret details below, then run the cell.
#@markdown ---
#@markdown Your GitHub username.
github_username = "" #@param {type:"string"}
#@markdown The name of the new repository you created on GitHub.
repository_name = "" #@param {type:"string"}
#@markdown Your email address for the Git commit.
user_email = "" #@param {type:"string"}
#@markdown The name of the Colab Secret where your GitHub PAT is stored.
secret_name = "GITHUB_PAT" #@param {type:"string"}


# --- Library Import for Secrets ---
from google.colab import userdata
import os
import shutil

# --- Git Push Logic ---
try:
    # Securely get the PAT from the Colab Secrets manager using the provided name
    personal_access_token = userdata.get(secret_name)

    if not all([github_username, repository_name, user_email]):
        raise ValueError("Please fill out your username, repository name, and email in the form.")

    if not personal_access_token:
        raise ValueError(f"Could not find the secret named '{secret_name}'. Please ensure it is set correctly in the Secrets (🔑) panel.")

    # --- NEW: Define paths for the cleaner workflow ---
    unpacked_content_path = "project_content"
    cloned_repo_path = f"{repository_name}" # The folder where the repo will be cloned

    if not os.path.exists(unpacked_content_path):
        raise FileNotFoundError(f"The '{unpacked_content_path}' directory was not found. Please run Step 2 successfully first.")

    # Clean up any old cloned repo directory
    if os.path.exists(cloned_repo_path):
        shutil.rmtree(cloned_repo_path)

    # --- NEW: Clone, Copy, Commit, and Push Workflow ---

    # 1. Clone the repository from GitHub
    print(f"📂 Cloning '{repository_name}' from GitHub...")
    repo_url = f"https://{personal_access_token}@github.com/{github_username}/{repository_name}.git"
    !git clone {repo_url} {cloned_repo_path}

    # 2. Copy the unpacked project files into the cloned repository
    # Using 'shutil.copytree' is robust for copying directory contents
    print("🚚 Copying project files into the repository...")
    shutil.copytree(unpacked_content_path, cloned_repo_path, dirs_exist_ok=True)

    # 3. Configure Git user, commit the changes, and push
    print("🔧 Preparing to commit and push changes...")

    # This is the cleaner, more standard sequence of commands
    git_commands = [
        f"cd {cloned_repo_path}",
        f"git config user.name '{github_username}'",
        f"git config user.email '{user_email}'",
        "git add .",
        "git commit -m 'Add project files from Google Drive'",
        "git push"
    ]

    # Execute the command chain
    !{" && ".join(git_commands)}

    print(f"\n✅ Success! Your project files have been pushed to the '{repository_name}' repository.")

    # 4. Clean up the temporary folders
    shutil.rmtree(unpacked_content_path)
    shutil.rmtree(cloned_repo_path)

except Exception as e:
    print(f"\n❌ An error occurred during the Git push process: {e}")